In [1]:
import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

hac_mo = '11'

## Load file

In [2]:
cauti_file =  '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CAUTI/CAUTI_Data_opt_for_SIRs_2016_11.pkl'
clabsi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CLABSI/CLABSI_Data_opt_for_SIRs_2016_11.pkl'

tdf, tdf1, tdf2 = 0, 0, 0
cauti_df, clabsi_df = 0, 0
mrsa_df, cdi_df = 0, 0

opt_df = 0
cauti_df = pd.read_pickle(cauti_file)
cauti_df['CAUTI SIS'] = (cauti_df['CAUTI Observed Cases'] - cauti_df['expected O'])/cauti_df['CAUTI Predicted Cases']
cauti_df.rename(columns={
        'O/E': 'CAUTI O/P',
        'simulated O': 'CAUTI simulated O',
        'simulated O/E': 'CAUTI simulated O/P',
        'expected O': 'CAUTI expected O',
        'expected O/E': 'CAUTI expected O/P',
        'pi_opt': 'CAUTI pi_opt',
        'z_opt': 'CAUTI z_opt',
    }, inplace=True)
cauti_sis_deciles = np.nanpercentile(cauti_df['CAUTI SIS'], np.arange(0, 100, 10))
cauti_rand_sir_deciles = np.nanpercentile(cauti_df['CAUTI expected O/P'], np.arange(0, 100, 10))

clabsi_df = pd.read_pickle(clabsi_file)
clabsi_df['CLABSI SIS'] = (clabsi_df['CLABSI Observed Cases'] - clabsi_df['expected O'])/clabsi_df['CLABSI Predicted Cases']
clabsi_df.rename(columns={
        'O/E': 'CLABSI O/P',
        'simulated O': 'CLABSI simulated O',
        'simulated O/E': 'CLABSI simulated O/P',
        'expected O': 'CLABSI expected O',
        'expected O/E': 'CLABSI expected O/P',
        'pi_opt': 'CLABSI pi_opt',
        'z_opt': 'CLABSI z_opt',
    }, inplace=True)
clabsi_sis_deciles = np.nanpercentile(clabsi_df['CLABSI SIS'], np.arange(0, 100, 10))
clabsi_rand_sir_deciles = np.nanpercentile(clabsi_df['CLABSI expected O/P'], np.arange(0, 100, 10))

ls1 = list(cauti_df)
ls2 = list(clabsi_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = cauti_df.merge(clabsi_df, on=ls, how='outer')
del cauti_df, clabsi_df

print('There are', len(df_yr['Facility ID'].unique()), 'hospitals in 2015 HACRP having SIS scores for CAUTI, CLABSI, or both. This excludes many that only had scores for PSI-90 or that were given max scores for failing to submit data without a proper exception.')


There are 2596 hospitals in 2015 HACRP having SIS scores for CAUTI, CLABSI, or both. This excludes many that only had scores for PSI-90 or that were given max scores for failing to submit data without a proper exception.


In [3]:
main_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.pkl')
main_df = main_df[main_df['file_year'] == '2016']
main_df.sort_values(by='Facility ID', inplace=True)
print('Initial no. of hospitals in main_df:', len(main_df['Facility ID'].unique()))

main_df = main_df[(~main_df['Total HAC Score'].isin([float("NaN"), np.nan]))]
print('No. of hospitals in main_df having real HAC scores:', len(main_df['Facility ID'].unique()))

main_df = main_df[main_df['State'] != 'MD']
print('No. of these hospitals not in Maryland:', len(main_df['Facility ID'].unique()))

main_df = main_df[(main_df['CAUTI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CAUTI Predicted Cases'] < 1)]
main_df = main_df[(main_df['CLABSI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CLABSI Predicted Cases'] < 1)]
main_df = main_df[~main_df['AHRQ PSI-90 Score'].isin([np.nan, float('NaN')])]
print('No. of these hospitals without legit predicted cases for CAUTI and CLABSI:', len(main_df['Facility ID'].unique()))

main_df.head()

Initial no. of hospitals in main_df: 3352
No. of hospitals in main_df having real HAC scores: 3243
No. of these hospitals not in Maryland: 3201
No. of these hospitals without legit predicted cases for CAUTI and CLABSI: 647


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
3191,641137,829246.0,1041149.0,4,010008,2016,1041149.0,1293654.0,12937.0,1280717.0,10411.49,NaN,6.0,NaN,NaN,6302014.0,6.0,7012012.0,2014-12-31,NaN,2013-01-01,CRENSHAW COMMUNITY HOSPITAL,2016.0,AL,6.0,11,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2016_11,AL,NaN,NaN,NaN,NaN,NaN,NaN,12028.500000,12937.0
3216,694270,388699.0,481911.0,2,010018,2016,440003.0,440003.0,4400.0,435603.0,4819.11,NaN,6.0,NaN,NaN,6302014.0,6.0,7012012.0,2014-12-31,NaN,2013-01-01,CALLAHAN EYE HOSPITAL,2016.0,AL,6.0,11,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2016_11,AL,NaN,NaN,NaN,NaN,NaN,NaN,3401.000000,4400.0
3235,645388,2009330.0,1932099.0,4,010022,2016,1932099.0,2334245.0,23342.0,2310903.0,19320.99,NaN,6.0,NaN,NaN,6302014.0,6.0,7012012.0,2014-12-31,NaN,2013-01-01,CHEROKEE MEDICAL CENTER,2016.0,AL,6.0,11,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2016_11,AL,NaN,NaN,NaN,NaN,NaN,NaN,15601.142857,23342.0
3259,651367,652071.0,641419.0,9,010032,2016,641419.0,809330.0,8093.0,801237.0,6414.19,NaN,5.0,NaN,NaN,6302014.0,5.0,7012012.0,2014-12-31,NaN,2013-01-01,WEDOWEE HOSPITAL,2016.0,AL,5.0,11,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,5.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [4]:
ls1 = list(df_yr)
ls2 = list(main_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = df_yr.merge(main_df, on=ls, how='outer')

print(df_yr.shape)
print(len(df_yr['Facility ID'].unique()), 'hospitals resulting from merging the above dataframes.')
print('This is the same number as the number of hospitals having real HAC scores.')

(3243, 116)
3243 hospitals resulting from merging the above dataframes.
This is the same number as the number of hospitals having real HAC scores.


## Generate SIS-based scores

In [31]:
hais = ['CAUTI', 'CLABSI']

for hai in hais:
    
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
        
        derived_sis = tdf2[hai + ' SIS'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, sis in enumerate(derived_sis):
            sis = float(sis)
            if np.isnan(sis) == True:
                derived_scores.append(np.nan)
            elif hai == 'CLABSI':
                score = np.nan
                if sis <= clabsi_sis_deciles[0]:
                    score = 1
                elif sis <= clabsi_sis_deciles[1]:
                    score = 2
                elif sis <= clabsi_sis_deciles[2]:
                    score = 3
                elif sis <= clabsi_sis_deciles[3]:
                    score = 4
                elif sis <= clabsi_sis_deciles[4]:
                    score = 5
                elif sis <= clabsi_sis_deciles[5]:
                    score = 6
                elif sis <= clabsi_sis_deciles[6]:
                    score = 7
                elif sis <= clabsi_sis_deciles[7]:
                    score = 8
                elif sis <= clabsi_sis_deciles[8]:
                    score = 9
                elif sis > clabsi_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan

                derived_scores.append(score)

            elif hai == 'CAUTI':
                score = np.nan
                if sis <= cauti_sis_deciles[0]:
                    score = 1
                elif sis <= cauti_sis_deciles[1]:
                    score = 2
                elif sis <= cauti_sis_deciles[2]:
                    score = 3
                elif sis <= cauti_sis_deciles[3]:
                    score = 4
                elif sis <= cauti_sis_deciles[4]:
                    score = 5
                elif sis <= cauti_sis_deciles[5]:
                    score = 6
                elif sis <= cauti_sis_deciles[6]:
                    score = 7
                elif sis <= cauti_sis_deciles[7]:
                    score = 8
                elif sis <= cauti_sis_deciles[8]:
                    score = 9
                elif sis > cauti_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan

                derived_scores.append(score)
            
        tdf2[hai + ' SIS-based score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    
    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' SIS-based score'] = 10
        df_yr = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_yr = tdf2.copy(deep=True)


len: 3233 3233
len: 3233 3233


## Generate scores based on random expectations

In [32]:
hais = ['CAUTI', 'CLABSI']

for hai in hais:
    
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
        
        derived = tdf2[hai + ' expected O/P'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, s in enumerate(derived):
            s = float(s)
            if np.isnan(s) == True:
                derived_scores.append(np.nan)
            
            elif hai == 'CLABSI':
                score = np.nan
                if s <= clabsi_rand_sir_deciles[0]:
                    score = 1
                elif s <= clabsi_rand_sir_deciles[1]:
                    score = 2
                elif s <= clabsi_rand_sir_deciles[2]:
                    score = 3
                elif s <= clabsi_rand_sir_deciles[3]:
                    score = 4
                elif s <= clabsi_rand_sir_deciles[4]:
                    score = 5
                elif s <= clabsi_rand_sir_deciles[5]:
                    score = 6
                elif s <= clabsi_rand_sir_deciles[6]:
                    score = 7
                elif s <= clabsi_rand_sir_deciles[7]:
                    score = 8
                elif s <= clabsi_rand_sir_deciles[8]:
                    score = 9
                elif s > clabsi_rand_sir_deciles[8]:
                        score = 10
                else:
                    score = np.nan

                derived_scores.append(score)

            elif hai == 'CAUTI':
                score = np.nan
                if s <= cauti_rand_sir_deciles[0]:
                    score = 1
                elif s <= cauti_rand_sir_deciles[1]:
                    score = 2
                elif s <= cauti_rand_sir_deciles[2]:
                    score = 3
                elif s <= cauti_rand_sir_deciles[3]:
                    score = 4
                elif s <= cauti_rand_sir_deciles[4]:
                    score = 5
                elif s <= cauti_rand_sir_deciles[5]:
                    score = 6
                elif s <= cauti_rand_sir_deciles[6]:
                    score = 7
                elif s <= cauti_rand_sir_deciles[7]:
                    score = 8
                elif s <= cauti_rand_sir_deciles[8]:
                    score = 9
                elif s > cauti_rand_sir_deciles[8]:
                    score = 10
                else:
                    score = np.nan

                derived_scores.append(score)
            
        tdf2[hai + ' random-based score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    
    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' random-based score'] = 10
        df_yr = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_yr = tdf2.copy(deep=True)
        
df_yr.head()

len: 3233 3233
len: 3233 3233


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/P,CAUTI simulated O,CAUTI simulated O/P,CAUTI expected O,CAUTI expected O/P,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/P,CLABSI simulated O,CLABSI simulated O/P,CLABSI expected O,CLABSI expected O/P,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,CAUTI SIS-based score,CLABSI SIS-based score,CAUTI random-based score,CLABSI random-based score
0,646010,19862213.0,23074556.0,2,670088,2016,23074556.0,23096349.0,230963.0,22865386.0,230745.56,NaN,9.0,5.0,8.0,6302014.0,9.0,7012012.0,2014-12-31,7.3333,2013-01-01,SCOTT & WHITE HOSPITAL- COLLEGE STATION,2016.0,TX,7.750,11,Yes,2082.0,1094.0,988.0,1.0,1.0,1.313,1.511,0.761615,0.661813,5.0,8.0,7.750,Yes,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230964.0,NaN,NaN,NaN,NaN,NaN,2016_11,TX,NaN,NaN,NaN,NaN,0.000433,1.0,264669.333333,230963.0,0.761615,0.0,0.000000,0.400622,0.305120,0.003734,10062.332492,0.456495,0.661813,0.0,0.000000,0.397431,0.263025,0.000973,1400.616209,0.398789,9.0,10.0,3.0,2.0
1,643070,8568276.0,10010897.0,2,670085,2016,10010897.0,10047173.0,100472.0,9946701.0,100108.97,NaN,4.0,3.0,1.0,6302014.0,4.0,7012012.0,2014-12-31,3.3333,2013-01-01,TEXAS HEALTH HARRIS METHODIST HOSPITAL ALLIANCE,2016.0,TX,3.500,11,No,2724.0,1435.0,1289.0,1.0,0.0,2.870,2.431,0.348432,0.000000,3.0,1.0,3.500,No,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2016_11,TX,NaN,NaN,NaN,NaN,NaN,NaN,111427.666667,100472.0,0.348432,0.0,0.000000,0.668849,0.233049,0.003734,10062.332492,0.115384,0.000000,2.0,0.822707,0.600771,0.247129,0.000973,1400.616209,-0.247129,7.0,4.0,2.0,2.0
2,642382,2083409

In [33]:
holdout_df = df_yr[(df_yr['State'] == 'MD') | (~df_yr['Payment Reduction'].isin(['Yes', 'No'])) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

print(holdout_df.shape[0])
print(df_yr.shape[0])
print(holdout_df.shape[0] + df_yr.shape[0])


42
3201
3243


## Generate SIS-based HAC scores and penalty assignments

In [34]:

hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
    d2_o = 0
    
    w_ls = []
    sum_ls = []

    # Use derived scores for CAUTI and CLABSI
    m_ls = ['CAUTI SIS-based score', 'CLABSI SIS-based score', 'SSI Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1 and np.isnan(v[0]) == False:
            print('len(list(set(v))) > 1')
            print(len(list(set(v))))
            print(list(set(v)))
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True:
            hac_scores.append(np.nan)
        
        elif np.isnan(d2) == False:
            hac_scores.append(d2)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        # If the derived score is NaN
        if np.isnan(d2) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float
        elif np.isnan(d2) == False:
            hac_scores.append(0.25*d1 + 0.75*d2)

    else:
        print(d1, ',', d2, ',', d2_o)
        break
        

        

df_yr['Total HAC Score (SIS-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')

p75 = np.percentile(df_yr['Total HAC Score (SIS-based)'], 75, method='linear')

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (SIS-based)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()

df_yr['Payment Reduction (SIS-based)'] = pr
            
o_list = df_yr['Payment Reduction'].tolist()
d_list = df_yr['Payment Reduction (SIS-based)'].tolist()

same = 0
diff = 0
p_to_np = 0
np_to_p = 0

res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        if o == 'Yes' and d_list[i] == 'No':
            p_to_np += 1
        elif o == 'No' and d_list[i] == 'Yes':
            np_to_p += 1
        else:
            print('Error')
            break
        res_ls.append(0)
               
print(same, "SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.")
print(diff, "SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.")
print(str(np.round(100 * same/(same+diff),2)) + '% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."\n')

print(p_to_np, 'hospitals were penalized but should not have been.')
print(np_to_p, 'hospitals were NOT penalized but should have been.')


3201 hospitals in hac_df
2947 SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.
254 SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.
92.06% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."

149 hospitals were penalized but should not have been.
105 hospitals were NOT penalized but should have been.


## Generate random-based HAC scores and penalty assignments

In [36]:

hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
    d2_o = 0
    
    w_ls = []
    sum_ls = []

    # Use derived scores for CAUTI and CLABSI
    m_ls = ['CAUTI random-based score', 'CLABSI random-based score', 'SSI Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1 and np.isnan(v[0]) == False:
            print('len(list(set(v))) > 1')
            print(len(list(set(v))))
            print(list(set(v)))
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True:
            hac_scores.append(np.nan)
        
        elif np.isnan(d2) == False:
            hac_scores.append(d2)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        # If the derived score is NaN
        if np.isnan(d2) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float
        elif np.isnan(d2) == False:
            hac_scores.append(0.25*d1 + 0.75*d2)

    else:
        print(d1, ',', d2, ',', d2_o)
        break
        

        

df_yr['Total HAC Score (random-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')

p75 = np.percentile(df_yr['Total HAC Score (random-based)'], 75, method='linear')

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (random-based)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()

df_yr['Payment Reduction (random-based)'] = pr
            

3201 hospitals in hac_df


In [37]:
## SIS-based

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (SIS-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(tdf2.shape[0], len(tdf2['Facility ID'].unique()))
print('\n')

## Random-based 
tdf1 = df_yr[df_yr['Payment Reduction (random-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (random-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (random-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(tdf2.shape[0], len(tdf2['Facility ID'].unique()))
print('\n')

## Original payment reduction
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(df_yr.shape[0], len(df_yr['Facility ID'].unique()))

0.21930646672914714
0.7806935332708529
3201 3201


0.24867228990940332
0.7513277100905967
3201 3201


0.2330521711965011
0.7669478288034989
3201 3201


In [38]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)

(3201, 124)
(42, 120)
(3243, 124)


In [39]:
p_np_df = df_yr[(df_yr['Payment Reduction'] == 'Yes') & (df_yr['Payment Reduction (SIS-based)'] == 'No')]
ip_ = np.round(np.nansum(p_np_df['HAC penalty, final']))
print(p_np_df.shape[0])

np_p_df = df_yr[(df_yr['Payment Reduction'] == 'No') & (df_yr['Payment Reduction (SIS-based)'] == 'Yes')]
is_ = np.round(np.nansum(np_p_df['HAC penalty, final']))
print(np_p_df.shape[0])

print(ip_, 'dollars of inappropriate penalties')
print(is_, 'dollars of inappropriate hospital savings')
print('', ip_ - is_)


149
105
89604876.0 dollars of inappropriate penalties
23486603.0 dollars of inappropriate hospital savings
 66118273.0


In [40]:
df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/finalized/final_2016.pkl', protocol=5)